In [37]:
from IPython.display import display

In [38]:
import re
import string
import scipy
import sklearn
import nltk
import os
import numpy as np
import codecs
import sys
import json
import random
import glob
from collections import Counter
from string import punctuation
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import collocations
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn import model_selection, svm, metrics
from sklearn.model_selection import train_test_split

In [39]:
customstopwords = set(line.strip() for line in open('stopword_ko.txt', 'r', encoding='utf-8'))
def removeStopwords(wordlist, customstopwords):
    return [word for word in wordlist if word not in customstopwords]

def isNumber(word):
    try:
        float(word)
        return True
    except ValueError:
        return False

In [40]:
# doc word frequency
def docWordFrequency(doc_line):
    doc_freq = dict()
    doc_words = doc_line.split()
    doc_wowords = removeStopwords(doc_words, customstopwords)
    for word in doc_wowords:
        count = doc_freq.get(word, 0)
        doc_freq[word] = count + 1
    return doc_freq

In [41]:
# get sentence word frequency
def getWordFrequency(tmp_list, doc_freq_dict):
    freq = 0
    for word in tmp_list:
#       print (word, "\t sentence freq: ", freq, "\t word freq : ", doc_freq_dict.get(word))
        freq = freq + doc_freq_dict.get(word)
    return freq

In [42]:
def FeatureWordFrequency (input_lines, doc_freq_dict):
    sent_freq = []
    for sentence in input_lines:
        temp_freq = docWordFrequency(sentence)
        temp_dict = Counter(temp_freq)
        doc_freq_dict = temp_dict + Counter(doc_freq_dict)
    
    for sentence in input_lines:
        tmp_list = sentence.split()
        tmp_list = removeStopwords(tmp_list, customstopwords)
        freq = getWordFrequency(tmp_list, doc_freq_dict)
        sent_freq.append(freq)
    
    return sent_freq

In [43]:
def FeatureTFIDF(input_lines):
    cv = TfidfVectorizer()
    x = cv.fit_transform(input_lines)
    sent_tfidf = x.sum(axis=1)
    #setn_tfidf = sum(setn_tfidf, [])
    array = np.array(sent_tfidf)
    array = array.flatten()
    return array

In [44]:
def FeatureNNP(input_lines):
    kkma = Kkma()
    nnp_list = []
    for sentence in input_lines:
        tmp_list = kkma.pos(sentence)
        count = 0
        for idx in range(len(tmp_list)):
            y = tmp_list[idx]
            if y[1] == 'NNP':
                count += 1
        nnp_list.append(count)
    return nnp_list

In [45]:
def FeatureWordCooccur(input_lines):
    wordcooccur = []
    for sentence in input_lines:
        bigram_measures = collocations.BigramAssocMeasures()
        trigram_measures = collocations.TrigramAssocMeasures()
        
        tmp_list = sentence.split()
        tmp_list = removeStopwords(tmp_list, customstopwords)
        
        finder_bigram = collocations.BigramCollocationFinder.from_words(tmp_list)
        finder_trigram = collocations.TrigramCollocationFinder.from_words(tmp_list)
        
        bigram_sent = finder_bigram.score_ngrams(bigram_measures.raw_freq)
        trigram_sent = finder_trigram.score_ngrams(trigram_measures.raw_freq)
        
        sum_bigram = 0
        sum_trigram = 0
        
        for idx in range(len(bigram_sent)):
            y = bigram_sent[idx]
            sum_bigram += y[1]
        for idx in range(len(trigram_sent)):
            y = trigram_sent[idx]
            sum_trigram += y[1]
        wordcooccur.append(sum_bigram+sum_trigram)
    return wordcooccur

In [10]:
def FeatureSentenceLenth(input_lines):
    sent_length = []
    tmp_length = []
    for sentence in input_lines:
        tmp_length.append(len(sentence))
    
    for idx in range(len(tmp_length)):
        sent_length.append(tmp_length[idx] / max(tmp_length))
    return sent_length

In [11]:
# 안씀
def FeatureCuePhrase(input_lines):
    cuephrase_text = open('cue_phrases_ko.txt', 'r') # cue-phrase text

In [12]:
def FeatureNumerical(input_lines):
    count_list = []
    for sentence in input_lines:
        tmp_list = sentence.split()
        for word in tmp_list:
            if isNumber(word):
                count_list.append(1)
            else:
                count_list.append(0)
    return count_list

In [13]:
def Feature_sentence_position(body_list):
    score_list = []
    for i in range(0, len(body_list)):
        score_list.append((len(body_list)-i)/len(body_list))
    return score_list

In [14]:
def Feature_sentence_centrality(body_list):
    score_list = []
    for index in range(0, len(body_list)):
        keywords = []
        other_keywords = []
        for sentence in body_list:
            if index == body_list.index(sentence):
                keywords = sentence.split(' ')
                keywords = removeStopwords(keywords, customstopwords)
            else:
                words = sentence.split(' ')
                words = removeStopwords(words, customstopwords)
                other_keywords += words
        
        molecular = 0
        denominator = 0
        
        for word in keywords:
            if word in other_keywords:
                molecular +=1
        
        denominator = len(keywords) + len(other_keywords) - molecular
        result = molecular/denominator
        score_list.append(result)
    return score_list
        
        

In [15]:
def Feature_sentence_resemblance_title(title, body_list):
    score_list = []
    title_words = title.split(' ')
    title_words = removeStopwords(title_words, customstopwords)
    
    for sentence in body_list:
        score = 0
        sentence = sentence.split(' ')
        sentence = removeStopwords(sentence, customstopwords)
        for word in title_words:
            if word in sentence:
                score+=1
        score /=len(title_words)
        score_list.append(score)
    
    return score_list
    

In [16]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Twitter()
        self.stopwords = customstopwords
        
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                        if noun not in self.stopwords and len(noun) > 1]))
        return nouns

In [17]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [18]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로 
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [19]:
class TextRank(object):
    def __init__(self, body_list):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences= body_list
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        print(self.nouns)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        
        self.word_rank_idx =  self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
        
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
        
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        
        return keywords
    def get_sent_rank_idx():
        return self.sent_rank_idx

In [20]:
def Feature_textrank(body_list):
    textrank = TextRank(body_list)
    sent_rank_idx = textrank.get_sent_rank_idx()
    score_list = []
    for index in range(0, len(sent_rank_idx)):
        score_list.append(sent_rank_idx[index])
    return score_list

In [21]:
def get_body_list(body):
    body_list = body.split('.')
    del_list=[]
    for index, sentence in enumerate(body_list):
        body_list[index] = sentence.strip()
        if body_list[index] == '':
            del_list.append(index)
    
    
    if len(del_list) != 0:
        count = 0
        for index in del_list:
            body_list.pop(index-count)
            count = count+1
    return body_list

In [22]:
def get_summary_list(summary):
    summary_list = summary.split('.')
    del_list=[]
    for index, sentence in enumerate(summary_list):
        summary_list[index] = sentence.strip()
        if summary_list[index] == '':
            del_list.append(index)
            
    if len(del_list) != 0:
        count = 0
        for index in del_list:
            summary_list.pop(index-count)
            count = count+1
    return summary_list

In [28]:
def makeX(title, body): #여기에 뭐들어가야되냐..?
    
    
    body_list = get_body_list(body)
    
    features = []
#     features.append(FeatureWordFrequency(body_list,docWordFrequency(body)))# 여기 빈칸에 뭐들어가야되냐..?
    features.append(FeatureTFIDF(body_list))
    features.append(FeatureNNP(body_list))
    features.append(FeatureWordCooccur(body_list))
    features.append(FeatureSentenceLenth(body_list))
    features.append(FeatureNumerical(body_list))
    features.append(Feature_sentence_position(body_list))
    features.append(Feature_sentence_centrality(body_list))
    features.append(Feature_sentence_resemblance_title(title, body_list))
#     features.append(Feature_textrank(body_list))

    X = [] # 한 문서당
    for index in range(0,len(body_list)):
        individual_x = [] # 한 문장당
        for feature in features:
            individual_x.append(feature[index])
        X.append(individual_x)
    return X
        

In [29]:
# summary_total = 0
# summary_n = 0
# summary_p = 0

def makeY(summary, body):
    
    
    summary_list = get_summary_list(summary)
    
    body_list = get_body_list(body)
    
    
    exist_indices=[]
    label=[]
    
    #각각의 문장이 문서마다 언제 끊기는지 알아야 할 때
    index_for_document=[]
    
    #summary가 모두 0일때 하니 summary이용할때 같이 하는게 좋음.
    zero_summary_count = 0
    len_summary_sentence = len(summary_list)
    total_document_count = 0
    zero_document_count = 0
    
    for sum_sentence in summary_list:
        if sum_sentence in body_list:
            exist_index = body_list.index(sum_sentence)
            exist_indices.append(exist_index)
        else:
            zero_summary_count+=1
            
    for i in range(0,len(body_list)):
        if i in exist_indices:
            label.append(1)
        else:
            label.append(-1)
            
    index_for_document = len(body_list)
    
    total_document_count = 1
    if 1 not in label:
        zero_document_count +=1
    
    sentence_document_params = [zero_summary_count, len_summary_sentence, total_document_count, zero_document_count]
    
    return exist_indices, label, index_for_document, sentence_document_params
            

In [30]:
#읽는 작업
if __name__ == "__main__":

    input_path = "wg_data/"

    X_before_embedding = []#임베딩 전 문서 
    X = []#전체 문서
    y = []#전체 문서
    sentence_for_filepath = []#어떤 문서 순으로?
    indice_for_document = []#문서별로 인덱스 표시
    sentence_document_params = []# 0인 summary와 ,전체 summary 개수, 전체 문서 개수, 전체 문서당 summary가 0인거 계산 아래 4개는 차례대로
    zero_summary_sentence_count = 0
    total_summary_sentence_count = 0
    total_document_count = 0
    zero_document_count = 0
    
    for input_file in glob.glob(os.path.join(input_path,'wg_data*.json')):
        json_data = open(input_file, 'r').read()
        data = json.loads(json_data)

        title = data['title']
        body = data['body']
        summary = data['summary']
        
        feature = makeX(title, body)
        exist_indices, label, index_for_document, sentence_document_params = makeY(summary, body)
        raw_X = get_body_list(body)
        
        X+=feature
        y+=label
        X_before_embedding+=raw_X
        
        #문서마다 바디 몇줄인지
        if len(indice_for_document) != 0:
            indice_for_document.append(indice_for_document[-1]+index_for_document)
        else:
            indice_for_document.append(index_for_document)
        
        #파일 순서 찾기
        sentence_for_filepath.append(input_file)
        
        #
        zero_summary_sentence_count += sentence_document_params[0]
        total_summary_sentence_count += sentence_document_params[1]
        total_document_count += sentence_document_params[2]
        zero_document_count += sentence_document_params[3]
    print(sentence_for_filepath)
    print("본문에 summary가 없는 summary 개수: ", zero_summary_sentence_count)
    print("모든 summary 개수: ", total_summary_sentence_count)
    print("모든 문서 개수: ", total_document_count)
    print("본문에 summary가 하나도 없는 문서 개수: ", zero_document_count)
    print(indice_for_document)
    print(X_before_embedding[0])

['wg_data/wg_data1551.json', 'wg_data/wg_data2948.json', 'wg_data/wg_data1997.json', 'wg_data/wg_data1100.json', 'wg_data/wg_data2140.json', 'wg_data/wg_data2036.json', 'wg_data/wg_data1643.json', 'wg_data/wg_data1579.json', 'wg_data/wg_data2626.json', 'wg_data/wg_data2080.json', 'wg_data/wg_data2752.json', 'wg_data/wg_data2426.json', 'wg_data/wg_data2576.json', 'wg_data/wg_data1473.json', 'wg_data/wg_data1399.json', 'wg_data/wg_data2285.json', 'wg_data/wg_data1084.json', 'wg_data/wg_data1521.json', 'wg_data/wg_data592.json', 'wg_data/wg_data595.json', 'wg_data/wg_data2538.json', 'wg_data/wg_data401.json', 'wg_data/wg_data2916.json', 'wg_data/wg_data2232.json', 'wg_data/wg_data1633.json', 'wg_data/wg_data653.json', 'wg_data/wg_data454.json', 'wg_data/wg_data2936.json', 'wg_data/wg_data1842.json', 'wg_data/wg_data2751.json', 'wg_data/wg_data2204.json', 'wg_data/wg_data776.json', 'wg_data/wg_data2157.json', 'wg_data/wg_data444.json', 'wg_data/wg_data2967.json', 'wg_data/wg_data907.json',

In [35]:


slice_num = int(len(indice_for_document)*(7/10))
slice_num2 = int(len(indice_for_document)*(2/10))
slice_num2 = slice_num + slice_num2
slice_num = indice_for_document[slice_num]
slice_num2 = indice_for_document[slice_num2]

X_train_total = X[0:slice_num2]
X_train = X[0:slice_num]
X_valid = X[slice_num:slice_num2]
X_test = X[slice_num2:]
y_train_total = y[0:slice_num2]
y_train = y[0:slice_num]
y_valid = y[slice_num:slice_num2]
y_test = y[slice_num2:]

# list_ = list(range(0,84779))
# X_train_total, X_test, y_train_total, y_test = train_test_split( X, y test_size=0.1,shuffle=False)
# X_train, X_valid, y_train, y_valid_ = train_test_split(X_train_total, y_train_total, test_size=0.22, shuffle=False)

# print(X_train)
# print(y_train)

clf = svm.SVC()
clf.fit(X_train,y_train)
    
valid_score = clf.score(X_valid,y_valid)

best_parameters={ 'C': clf.get_params()['C'], 'gamma': clf.get_params()['gamma']}

new_svm = svm.SVC(**best_parameters)
new_svm.fit(X_train_total,y_train_total)

predict = new_svm.predict(X_test)
accuracy = metrics.accuracy_score(y_test,predict)

one_label=[]
print("predict:")
for index in range(0,len(predict)):
    if predict[index]!=-1:
        print(X_test[index])
        print(predict[index])
        one_label.append(X_before_embedding[slice_num2+index])
    print()
print()
print("accuracy: ", accuracy)

print("label이 1인 predict의 원래 문장:")
print(one_label)

predict:
[3.6060507346765704, 1, 1.9032258064516117, 1.0, 0, 1.0, 0.09322033898305085, 0.0]

[2.198454232949964, 0, 1.4000000000000001, 0.1981981981981982, 0, 0.9444444444444444, 0.0078125, 0.0]

[3.1316516850796075, 0, 1.7999999999999998, 0.6756756756756757, 0, 0.8888888888888888, 0.04878048780487805, 0.0]

[3.5755008988540626, 1, 1.8695652173913035, 0.8558558558558559, 1, 0.8333333333333334, 0.06611570247933884, 0.0]

[2.7933872938902833, 0, 1.7857142857142851, 0.5045045045045045, 0, 0.7777777777777778, 0.023809523809523808, 0.0]

[3.8401715735846227, 0, 1.833333333333334, 0.7207207207207207, 0, 0.7222222222222222, 0.044534412955465584, 0.0]

[2.6078365718960588, 1, 1.7272727272727275, 0.4144144144144144, 1, 0.6666666666666666, 0.023809523809523808, 0.0]

[2.605328260898971, 0, 1.7272727272727275, 0.4144144144144144, 0, 0.6111111111111112, 0.027888446215139442, 0.0]

[2.825947970221391, 0, 1.75, 0.43243243243243246, 0, 0.5555555555555556, 0.019762845849802372, 0.0]

[3.26509075067788

[4.052102030010734, 1, 1.9166666666666674, 0.8, 0, 1.0, 0.1259259259259259, 0.2]

[4.955757983909091, 0, 1.9375000000000013, 1.0, 1, 0.8571428571428571, 0.22580645161290322, 0.0]

[3.52852826332699, 0, 1.823529411764706, 0.375, 0, 0.7142857142857143, 0.08571428571428572, 0.0]

[4.084697652833781, 0, 1.8571428571428579, 0.48, 0, 0.5714285714285714, 0.05555555555555555, 0.0]

[4.517444073224432, 0, 1.8888888888888877, 0.625, 0, 0.42857142857142855, 0.08571428571428572, 0.0]

[1.4142135623730951, 0, 0.5, 0.115, 0, 0.2857142857142857, 0.0, 0.0]

[1.0, 0, 0, 0.015, 0, 0.14285714285714285, 0.0, 0.0]

[4.30101264270616, 0, 1.8928571428571423, 0.5497630331753555, 0, 1.0, 0.030425963488843813, 0.0]

[2.8138399929378908, 0, 1.8125, 0.2890995260663507, 0, 0.9666666666666667, 0.026262626262626262, 0.0]

[3.95748634717605, 0, 1.8846153846153837, 0.5497630331753555, 0, 0.9333333333333333, 0.030425963488843813, 0.0]

[3.7795754728982587, 0, 1.8888888888888877, 0.5781990521327014, 0, 0.9, 0.0346232179

[2.6298923848862965, 0, 1.5714285714285712, 0.1471861471861472, 0, 0.9545454545454546, 0.013262599469496022, 0.0]

[4.503663068894351, 1, 1.8965517241379302, 0.6017316017316018, 0, 0.9090909090909091, 0.05817174515235457, 0.0]

[3.171025537661944, 0, 1.75, 0.2813852813852814, 0, 0.8636363636363636, 0.018666666666666668, 0.25]

[1.4142135623730951, 0, 1.0, 0.03896103896103896, 0, 0.8181818181818182, 0.0026246719160104987, 0.0]

[3.567805994452991, 0, 1.823529411764706, 0.30303030303030304, 0, 0.7727272727272727, 0.018666666666666668, 0.0]

[3.08201751110329, 1, 1.769230769230769, 0.30303030303030304, 0, 0.7272727272727273, 0.0213903743315508, 0.25]

[4.564223811580106, 2, 1.8846153846153837, 0.4675324675324675, 0, 0.6818181818181818, 0.03804347826086957, 0.25]

[4.244670499235269, 0, 1.8500000000000005, 0.4458874458874459, 0, 0.6363636363636364, 0.032432432432432434, 0.25]

[4.613523570124007, 0, 1.8749999999999991, 0.5194805194805194, 0, 0.5909090909090909, 0.029649595687331536, 0.0]



[1.7114275482800383, 0, 1.5, 0.10810810810810811, 1, 0.8666666666666667, 0.00411522633744856, 0.1111111111111111]

[3.854395261825668, 0, 1.8965517241379302, 0.722972972972973, 0, 0.8, 0.07964601769911504, 0.0]

[4.341480779674474, 0, 1.9, 0.8513513513513513, 1, 0.7333333333333333, 0.07017543859649122, 0.1111111111111111]

[3.3892397003132495, 0, 1.8125, 0.4594594594594595, 0, 0.6666666666666666, 0.03829787234042553, 0.1111111111111111]

[2.4056453047669297, 0, 1.625, 0.22972972972972974, 1, 0.6, 0.016666666666666666, 0.0]

[4.09764014801554, 0, 1.8846153846153837, 0.722972972972973, 1, 0.5333333333333333, 0.06086956521739131, 0.0]

[2.4464127218400407, 0, 1.7272727272727275, 0.34459459459459457, 0, 0.4666666666666667, 0.012448132780082987, 0.0]

[3.295331019827711, 0, 1.833333333333334, 0.6148648648648649, 1, 0.4, 0.02092050209205021, 0.0]

[2.6026251357463757, 0, 1.6999999999999997, 0.3108108108108108, 0, 0.3333333333333333, 0.029535864978902954, 0.0]

[2.643819511166811, 0, 1.625, 0


[1.7320508075688772, 0, 1.625, 0.1527777777777778, 0, 0.13157894736842105, 0.007712082262210797, 0.0]

[1.0, 0, 0.5, 0.05555555555555555, 0, 0.10526315789473684, 0.0025575447570332483, 0.0]

[0.0, 0, 1.0, 0.034722222222222224, 0, 0.07894736842105263, 0.007712082262210797, 0.0]

[1.0, 0, 0.5, 0.08333333333333333, 0, 0.05263157894736842, 0.0025575447570332483, 0.0]

[4.078418681340778, 1, 1.8749999999999991, 0.5625, 0, 0.02631578947368421, 0.028871391076115485, 0.0]

[3.247878102455705, 0, 1.833333333333334, 0.3855421686746988, 0, 1.0, 0.03865979381443299, 0.2]

[2.576267099963906, 0, 1.769230769230769, 0.29518072289156627, 0, 0.9666666666666667, 0.02544529262086514, 0.0]

[2.614872662953682, 2, 1.625, 0.21084337349397592, 0, 0.9333333333333333, 0.015113350125944584, 0.0]

[3.398931775711945, 0, 1.7857142857142851, 0.3614457831325301, 0, 0.9, 0.02544529262086514, 0.0]

[3.1011131969798513, 0, 1.7999999999999998, 0.3795180722891566, 0, 0.8666666666666667, 0.02544529262086514, 0.2]

[4.69

[3.2813312090321687, 0, 1.8695652173913035, 0.5644171779141104, 0, 0.2903225806451613, 0.02962962962962963, 0.0]

[2.61436367860105, 0, 1.6999999999999997, 0.27607361963190186, 0, 0.25806451612903225, 0.016453382084095063, 0.0]

[3.1675729192658837, 0, 1.666666666666667, 0.3006134969325153, 0, 0.22580645161290322, 0.0054249547920434, 0.14285714285714285]

[1.963526546973389, 0, 1.25, 0.147239263803681, 0, 0.1935483870967742, 0.007246376811594203, 0.0]

[4.724086809877674, 1, 1.9, 0.754601226993865, 0, 0.16129032258064516, 0.02962962962962963, 0.0]

[3.424954772496041, 0, 1.769230769230769, 0.37423312883435583, 0, 0.12903225806451613, 0.012750455373406194, 0.0]

[1.4008936675465642, 0, 1.5, 0.15337423312883436, 0, 0.0967741935483871, 0.0054249547920434, 0.14285714285714285]

[3.6505371461955343, 0, 1.823529411764706, 0.43558282208588955, 0, 0.06451612903225806, 0.02962962962962963, 0.2857142857142857]

[3.600816067965033, 0, 1.823529411764706, 0.3496932515337423, 0, 0.03225806451612903,